In [2]:
# Code adapted from: https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/autonomousSequence.py

import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# CrazyFlie imports:
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

## Helper functions

In [3]:
group_number = 14

# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']

    print(x, y, z)
    return x, y, z


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


# Ascend and hover at 1m:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(5):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    return


# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index


# Find contours in the image
def check_contours(frame):

    print('Checking image:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)


    if largest_area > 100:
        return True
    else:
        return False


# Follow the setpoint sequence trajectory:
def adjust_position(cf, current_y, direction, change_direction):

    print('Adjusting position')

    steps_per_meter = int(10)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    for i in range(3):
        current_y = current_y - direction /float(steps_per_meter) if change_direction else current_y
        position = [current_x + 0.1, current_y, 0.5, 0.0] # x,y,z,yaw

        print('Setting position {}'.format(position))
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    time.sleep(0.1)
    return current_y


# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending:')
    # Hover at 0.5 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

# Find contours in the image
def check_contours_new(frame):

    print('Checking image:')

    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    # lb1 = (145, 35, 75)
    # ub1 = (180, 255, 255)
    # lb2 = (0, 75, 75)
    # ub2 = (20, 255, 255)
    lb1 = (160, 35, 50)  # Lower bound for red (higher hue range)
    ub1 = (180, 255, 255)  # Upper bound for red (higher hue range)
    lb2 = (0, 50, 50)  # Lower bound for red (lower hue range)
    ub2 = (20, 255, 255)  # Upper bound for red (lower hue range)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    largest_area, largest_contour_index = findGreatesContour(contours)



    return contours, largest_contour_index, largest_area

In [4]:
def getMask(frame):

    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (20, 255, 255)


    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    return mask

def getMaxBox(contours):
    max_area = 0
    max_box = ()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > 4000 and h > 175: # filter out small boxes
            print("area = ", w*h)
            center_x = x + w // 2
            center_y = y + h // 2

            if w*h > max_area:
                max_area = w*h
                max_box = (x, y, w, h, center_x, center_y)

    return max_box

def getMaxBlueBox(contours):
    max_area = 0
    max_box = ()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w * h > 1000 and h > 40: # filter out small boxes
            print("area = ", w*h)
            center_x = x + w // 2
            center_y = y + h // 2

            if w*h > max_area:
                max_area = w*h
                max_box = (x, y, w, h, center_x, center_y)

    return max_box



In [5]:
def getBlueMask(frame):
    # lower_blue1 = np.array([100, 50, 50])
    # upper_blue1 = np.array([130, 255, 255])

    # lower_blue2 = np.array([160, 50, 50])
    # upper_blue2 = np.array([180, 255, 255])

    # hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # # Create masks for each range
    # mask1 = cv2.inRange(hsv1, lower_blue1, upper_blue1)
    # mask2 = cv2.inRange(hsv2, lower_blue2, upper_blue2)

    # # Combine the masks
    # mask = cv2.bitwise_or(mask1, mask2)

    lb1 = (100, 50, 50)
    ub1 = (130, 255, 255)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lb1, ub1)

    return mask


In [6]:
# # test blue filter
# camera = 0
# cap = cv2.VideoCapture(camera)

# while(cap.isOpened()):
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     if not ret:
#         print('nothing')
#         continue

#     mask = getBlueMask(frame)

#     # Compute
#     cv2.imshow('mask', mask)

#     # Hit q to quit.
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the capture
# cap.release()
# cv2.destroyAllWindows()

## Controls + Flying

In [7]:
#TODO: add step sizes for smoother movement
    # calculate the step_amplitude
def step_amplitude(center_x: float):
    return min(0.5, 1-(((center_x/320 - 1))**2))

def controller(cf, max_box, x_cur, y_cur):
    if not max_box:
        print("No detection, just move forward")
        y_command = y_cur
        x_command = x_cur
        total_dx = 0.15
        iterations = 4
        # for i in range(3):
        #     x_command = x_command + 1.0/float(steps_per_meter)


        for _ in range(iterations):
            x_command += total_dx / iterations
            position = [x_command, y_command, 0.5, 0.0]
            for i in range(1):
                cf.commander.send_position_setpoint(position[0],
                                                    position[1],
                                                    position[2],
                                                    position[3])
                time.sleep(0.1)
        time.sleep(0.1)

        print('Setting position {}'.format(position))
        return False, x_command, y_command


    # window dimensions: X = 640, Y = 480. total area: 307,200
    x, y, w, h, center_x, center_y = max_box

    # Check relative location of the object
    isRight = 1
    if center_x < 320: # if the object is in the left part of the camera
        isRight = -1

    # Calculate drone position relative to object's position in camera
    step_gain = 0.1 # approx 4 cm
    step_amp = step_amplitude(center_x) # check this
    dy = step_gain * (isRight * step_amp)
    dx = (step_gain * (1 - step_amp))

    print("dx = ", dx, "dy = ", dy, "step_amp = ", step_amp, "center_x = ",  center_x)
    print("x_cur = ", x_cur,"y_cur = ", y_cur)

    y_command = y_cur + dy
    y_command = np.clip(y_command, -0.87, 0.87) # accounts for out-of-bounds
    if y_command == -0.87: # right edge
        print("hitting right edge! moving in")
        y_command = y_cur + 0.75*abs(dy)
    elif y_command == 0.87: # left edge
        print("hitting left edge! moving in")
        y_command = y_cur - 0.75*abs(dy)


    x_command = x_cur + dx

    print("new_x = ", x_command,"new_y = ", y_command)

    position = [x_command, y_command, 0.5, 0]

    print('Setting position {}'.format(position))
    for i in range(1):
        cf.commander.send_position_setpoint(position[0],
                                            position[1],
                                            position[2],
                                            position[3])
        time.sleep(0.1)


    return False, x_command, y_command


In [8]:

def object_controller(cf, max_box, x_cur, y_cur, frame, state):
    x_gain = 0
    y_gain = 0
    final = False

    if not max_box:
        print("Object not detected")
        if not state:
            if y_cur < 0: # drone is on the right, need to move left towards the center
                y_command = y_cur + 0.05
            else:
                y_command = y_cur - 0.05
        else: 
            y_command = y_cur + state * 0.05   # state is 1 if object is on the left, state is -1 if object is on the right 
        print("Setting y_cur = ",y_cur)
        x_command = x_cur
        for _ in range(1):
            cf.commander.send_position_setpoint(x_cur, y_command, 0.5, 0)
            time.sleep(0.1)

        return False, x_command, y_command, final, state
    
    box_x, box_y, box_width, box_height,center_x,center_y = max_box
        
    # this only happens once, upon first object detection 
    state = 1 if center_x < 320 else -1

    if center_x > 320 - 40 and center_x < 320 + 40: # centered, go back to main loop
        final = True 
        return False, x_cur, y_cur, final, state
    else:
        # Check relative location of the object
        isRight = 1
        if center_x > 320: # if the object is in the right part of the camera
            isRight = -1
        y_gain = isRight * 0.07 # originally 0.1

    x_command = x_cur + x_gain
    y_command = y_cur + y_gain

    cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
    cv2.rectangle(frame, (box_x, box_y), (box_x + box_width, box_y + box_height), (255, 0, 255), 2)

    position = [x_command, y_command, 0.5, 0]
    print('Setting position {}'.format(position))

    # Set velocity
    for i in range(1):
        cf.commander.send_position_setpoint(position[0],
                                            position[1],
                                            position[2],
                                            position[3])
        time.sleep(0.1)

    return False, x_command, y_command, final, state



    # print("Moving laterally towards object")
    # lateral_displacement = y_cur - center_x
    # position = [x_cur, lateral_displacement,0.5,0]

    # for i in range(10): # temporarly fixing step size at 10
    #     cf.commander.send_position_setpoint(position[0],
    #                                         position[1],
    #                                         position[2],
    #                                         position[3])
    #     time.sleep(0.05)
    # y_command = 0

    # # now move forward
    # while x_command < 3.50: # while less than the 3.5 meter mark
    #     x_command = x_cur + 0.02
    #     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)

In [9]:
# def sweep(cf, x_cur, y_cur, sweep_direction):

#     y_command = y_cur + sweep_direction * 0.05

#     # sam added this loop

#     for _ in range(5):
#         cf.commander.send_position_setpoint(x_cur, y_command, 0.5, 0)
#         time.sleep(0.05)
    
#     return False, y_command


In [10]:
# NOT TESTED YET
def avg_red_x_center(frame):
    mask = getMask(frame)
    coords = np.where(mask > 0)
    x_values = coords[1]
    return np.mean(x_values)

In [11]:
# NOT TESTED YET
def initialize_left_or_right(cf, frame, x_cur, y_cur):
    '''
    takes in the controller, the frame, and x_cur, and y_cur
    decides if the drone should start by moving to the right or the left
    depending on if there are more obstacles to the right or the left of
    the course
    '''
    red_avg = avg_red_x_center(frame)

    # Check relative location of the object
    isRight = 1
    if red_avg < 320: # if the object is in the left part of the camera
        isRight = -1
    print("direction=", isRight)
    # Calculate drone position relative to object's position in camera
    dy = 0.05 * isRight  # just picked 0.5 off a guess, should change
    dx = 0

    print("dx = ", dx, "dy = ", dy)
    print("x_cur = ", x_cur,"y_cur = ", y_cur)

    y_command = y_cur + dy
    x_command = x_cur


    # going in increments so it is smoother
    for _ in range(4):
        y_command = y_command + dy # 5 steps of 0.1, going to 0.5 meters left or right

        position = [x_command, y_command, 0.5, 0]
        for _ in range(1): # check that the drone gets there, 10 checks
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    print('Setting position {}'.format(position))
    print("new_x = ", x_command,"new_y = ", y_command)
    return x_command, y_command

## LET IT FLY

In [12]:
# Set the URI the Crazyflie will connect to
camera = 0

uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        x_cur = 0
        y_cur = 0
        exit_loop = False
        final = False
        start = True
        state = None

        cap = cv2.VideoCapture(camera)
        while(cap.isOpened()) and not exit_loop:

            ret, frame = cap.read()

            elapsed = time.time() - t
            if(elapsed > 7.0):

                print('Capturing.....')

                if ret:
                    #cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                        continue


                    # at the start, move to left or right a little
                    # NOT TESTED YET
                    if start:
                        print("We're starting")
                        x_cur, y_cur = initialize_left_or_right(cf, frame, x_cur, y_cur)
                        start = False
                        continue

                    # Switch to blue filter detection once drone is 3.2 meters away from start
                    if x_cur > 3.6:
                        if final:
                            print("Object detection y pos final, moving forward...")
                            for _ in range(3):
                                x_cur += 0.05
                                cf.commander.send_position_setpoint(x_cur, y_cur, 0.5, 0)
                                time.sleep(0.1)
                            print("YIPPEEEEEEEEE!!! :D")
                            break

                        print("Looking for object...")
                        mask = getBlueMask(frame)
                        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        max_box = getMaxBlueBox(contours)
                        # if not max_box:
                        #     print("Object not detected")
                        #     exit_loop, y_cur = sweep(cf, x_cur, y_cur) # this needs to sweep
                        #     continue
                        exit_loop, x_cur, y_cur, is_final, state = object_controller(cf, max_box, x_cur, y_cur,frame, state)
                        final = final or is_final

                    else:
                        mask = getMask(frame)
                        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                        max_box = getMaxBox(contours)
                        if not max_box:
                            cv2.putText(frame, "no detection", (10,10),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)
                            cv2.imshow('Frame',frame)
                            exit_loop, x_cur, y_cur = controller(cf, None, x_cur, y_cur)
                            if cv2.waitKey(10) & 0xFF == ord('q'):
                                break
                            continue
                        exit_loop, x_cur, y_cur = controller(cf, max_box, x_cur, y_cur)

                    
                    if not max_box:
                        print('not found')
                        continue
                    
                    
                    x, y, w, h, center_x, center_y = max_box

                    cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)

                    delta = int(w)
                    if center_x < 320:
                        del_center = str(center_x + delta) + "," + str(center_y)
                        cv2.circle(frame, (center_x + delta, center_y), 5, (255, 165, 0), -1)
                        cv2.putText(frame, del_center, (center_x + delta, center_y + 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)
                    else:
                        del_center = str(center_x - delta) + "," + str(center_y)
                        cv2.circle(frame, (center_x - delta, center_y), 5, (255, 165, 0), -1)
                        cv2.putText(frame, del_center, (center_x - delta, center_y + 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)
                    cv2.imshow('Frame', frame)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break

            # if(x_cur >= 3.5):
            #     hover_and_descend(cf)
            #     break

        # Descend and stop all motion:
        hover_and_descend(cf)

        cap.release()



print('Done!')

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/14/2M


[ WARN:0@2.798] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2024-12-13 09:54:33.491 python[68503:15522254] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Capturing.....
Initializing PID Controller
Capturing.....
We're starting
direction= -1
dx =  0 dy =  -0.05
x_cur =  0 y_cur =  0
Setting position [0, -0.25, 0.5, 0]
new_x =  0 new_y =  -0.25
Capturing.....
area =  4956
area =  13392
area =  13719
dx =  0.05 dy =  0.05 step_amp =  0.5 center_x =  387
x_cur =  0 y_cur =  -0.25
new_x =  0.05 new_y =  -0.2
Setting position [0.05, -0.2, 0.5, 0]
Capturing.....
area =  4860
area =  10336
area =  9555
dx =  0.05 dy =  0.05 step_amp =  0.5 center_x =  375
x_cur =  0.05 y_cur =  -0.2
new_x =  0.1 new_y =  -0.15000000000000002
Setting position [0.1, -0.15000000000000002, 0.5, 0]
Capturing.....
area =  5760
area =  12150
area =  11466
dx =  0.05 dy =  0.05 step_amp =  0.5 center_x =  370
x_cur =  0.1 y_cur =  -0.15000000000000002
new_x =  0.15000000000000002 new_y =  -0.10000000000000002
Setting position [0.15000000000000002, -0.10000000000000002, 0.5, 0]
Capturing.....
area =  6006
area =  12878
area =  12232
dx =  0.05 dy =  0.05 step_amp =  0.5

KeyboardInterrupt: 

In [13]:
cap.release()

## Test object detection only (no flying)

In [ ]:
camera = 0
cap = cv2.VideoCapture(camera)

if not cap.isOpened():
    print("Error: Unable to open video capture.")
    exit()


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        continue

    mask = getMask(frame)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_box = getMaxBox(contours)
    if not max_box:
        continue
    x, y, w, h, center_x, center_y = max_box
    cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)
    coords = str(int(w)) + "," + str(int(w*h))
    cv2.putText(frame, coords, (int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    centers = str(center_x) + "," + str(center_y)

    delta = int(w)
    if center_x < 320:
        del_center = str(center_x + delta) + "," + str(center_y)
        cv2.circle(frame, (center_x + delta, center_y), 5, (255, 165, 0), -1)
        cv2.putText(frame, del_center, (center_x + delta, center_y + 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)
    else:
        del_center = str(center_x - delta) + "," + str(center_y)
        cv2.circle(frame, (center_x - delta, center_y), 5, (255, 165, 0), -1)
        cv2.putText(frame, del_center, (center_x - delta, center_y + 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 165, 0), 2)


    # Compute
    cv2.imshow('mask', mask)
    cv2.imshow('Frame with Bounding Boxes', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

## OLD CODE

In [ ]:
# Set the URI the Crazyflie will connect to
camera = 0

uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_y = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        cap = cv2.VideoCapture(camera)
        while(cap.isOpened()):

            ret, frame = cap.read()

            elapsed = time.time() - t
            if(elapsed > 2.0):

                print('Capturing.....')

                if ret:
                    #cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                        continue

                    contours, largest_contour_index, largest_area = check_contours_new(frame)
                    # make bounding box
                    x,y,w,h = cv2.boundingRect(contours[largest_contour_index])
                    cv2.rectangle(frame, (int(x), int(y)), (int(w), int(h)), color, thickness=2)
                    print(int(x), int(y),int(w),int(h))
                    coords = str(int(x))+","+str(int(y))
                    cv2.putText(frame, coords, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                    if largest_area > 1000:
                        cv2.putText(frame, str(largest_area), (50,500),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                        print(largest_area, coords)
                    if (x > 100) and largest_area > 1000:
                        cv2.putText(frame, "move left", (50,200),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                        current_y = adjust_position(cf, current_y, direction = 1.0, change_direction=True)
                    elif (x <= 400) and largest_area > 1000:
                        cv2.putText(frame, "move right", (50,200),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                        current_y = adjust_position(cf, current_y, direction = -1.0, change_direction=True)
                    else:
                        current_y = adjust_position(cf, current_y, direction = -1.0, change_direction=False)


                    cv2.imshow('img', frame)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break

            if(elapsed > 120.0):
                        break

        cap.release()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')

In [ ]:
# OLD MAX BOX CODE
    # bounding_boxes = []
    # max_area = 0
    # max_box = []
    # for contour in contours:
    #     x, y, w, h = cv2.boundingRect(contour)
    #     if w * h > 4000 and h > 175: # filter out small boxes
    #         print("area = ", w*h)
    #         center_x = x + w // 2
    #         center_y = y + h // 2

    #         bounding_boxes.append((x, y, w, h, center_x, center_y))

    #         if w*h > max_area:
    #             max_area = w*h
    #             max_box = bounding_boxes[-1]
    #         # Draw rectangle in purple (BGR: 255, 0, 255)
    #         # cv2.circle(frame, (center_x, center_y), 5, (0, 255, 0), -1)
    #         # cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)
    #         # coords = str(int(h)) + "," + str(int(w*h))
    #         # cv2.putText(frame, coords, (int(x),int(y)),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
    #         # centers = str(center_x) + "," + str(center_y)
    #         # cv2.putText(frame, coords, (int(center_x),int(center_y)+5),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

# OLD DISTANCE CODE
    # bounding_boxes = sorted(bounding_boxes, key=lambda x: x[0])

    # # Compute distances between adjacent bounding boxes
    # max_dist = 0
    # indices=()
    # for i in range(len(bounding_boxes) - 1):
    #     center_x1 = bounding_boxes[i][4]  # x-coordinate of center of current bounding box
    #     center_y1 = bounding_boxes[i][5]  # y-coordinate of center of current bounding box
    #     center_x2 = bounding_boxes[i + 1][4]  # x-coordinate of center of next bounding box
    #     center_y2 = bounding_boxes[i + 1][5]

    #     distance = np.sqrt((center_x2 - center_x1) ** 2 + (center_y2 - center_y1) ** 2)

    #     if distance > max_dist:
    #         max_dist = distance
    #         indices = (i, i+1)
    #     #distances.append(abs(x2 - x1))

    # if indices:
    #     i1, i2 = indices
    #     x1,y1,w1,h1,cx1,cy1 = bounding_boxes[i1]
    #     x2,y2,w2,h2,cx2,cy2 = bounding_boxes[i2]
    #     cv2.line(frame, (cx1, cy1), (cx2,cy2), (255, 165, 0), 2)